In [40]:
import re
import numpy as np
import pandas as pd
from math import * 
import scipy as sp
from scipy.stats import mode
from sklearn import linear_model
import matplotlib
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
%matplotlib inline

In [41]:
time_data = pd.read_csv('../Source/2018_calendar.csv', delimiter=',')


In [42]:
time_data['price'] = time_data['price'].replace('[\$,)]','', regex=True).replace('[(]','-', regex=True).astype(float)
time_data['log_price'] = np.log(time_data['price'].values)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [43]:

time_data

,listing_id,date,available,price,log_price
0,16457286,2018-01-11,f,NaN,NaN
1,16457286,2018-01-12,f,NaN,NaN
2,16457286,2018-01-13,f,NaN,NaN
3,16457286,2018-01-14,f,NaN,NaN
4,16457286,2018-01-15,f,NaN,NaN
5,16457286,2018-01-16,f,NaN,NaN
6,16457286,2018-01-17,f,NaN,NaN
7,16457286,2018-01-18,f,NaN,NaN
8,16457286,2018-01-19,f,NaN,NaN
9,16457286,2018-01-20,f,NaN,NaN


In [44]:
time_data.size

88731500

In [45]:
# Only consider data for which there is data
nan_adjusted = time_data[np.isfinite(time_data['log_price'])]

In [46]:
#Still left with a reasonable amount of observations
nan_adjusted.size

30475005

In [8]:

# Group by listing_id, then find each listing id's average price
grouped = nan_adjusted.groupby(['listing_id'], as_index = False)

means = grouped['price'].mean().values
log_means = grouped['log_price'].mean().values

In [9]:
mean_df = grouped.mean()
mean_df['listing_id']

0            2515
1            2539
2            2595
3            3330
4            3647
5            3831
6            4611
7            4989
8            5054
9            5121
10           5135
11           5178
12           5238
13           5295
14           5441
15           5803
16           5945
17           6090
18           6848
19           6990
20           7097
21           7241
22           7322
23           7748
24           7801
25           7816
26           8024
27           8025
28           8110
29           8225
           ...   
31785    22656685
31786    22656871
31787    22656994
31788    22657120
31789    22657161
31790    22658582
31791    22659217
31792    22659796
31793    22660201
31794    22664180
31795    22664800
31796    22665286
31797    22665392
31798    22665675
31799    22666216
31800    22666545
31801    22666569
31802    22667087
31803    22667161
31804    22667271
31805    22667295
31806    22667496
31807    22667649
31808    22667815
31809    2

In [10]:

# Maybe I need to reset the index, who the hell knows if I'll use this later
nan_adjusted.reset_index(inplace=True)

In [11]:
# Initialize an empty column for the price differences 
nan_adjusted['price_diff'] = 0

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
new

In [47]:
for i in range(nan_adjusted['listing_id'].count()):
    nan_adjusted.iloc[i]['price_diff'] = nan_adjusted.iloc[i]['price'] - mean_df.iloc[nan_adjusted.iloc[i]['listing_id']]['price']

IndexError: single positional indexer is out-of-bounds

In [13]:
mean_df_new = mean_df[['listing_id','price']]

In [14]:
nan_adjusted = time_data[np.isfinite(time_data['log_price'])]

In [15]:
new = nan_adjusted.merge(mean_df_new,how='left', left_on='listing_id', right_on='listing_id')

In [16]:

# Compute price differences 
new['price_diff'] = new['price_x'] - new['price_y']

In [24]:
# Convert dates to datetime
dates = pd.to_datetime(new['date'].values)

In [25]:
# Add new datetime object
new['datetime'] = dates

In [26]:
# Groupby datetime, get average price difference
final = new.groupby(['datetime'], as_index = False)
final_sums = final['price_diff'].mean()

In [33]:
print ("Number of days average price difference was positive:"), sum(final_sums['price_diff']>0)

Number of days average price difference was positive:


(None, 250)

In [34]:
# Examine overall price difference trend

fig, ax = plt.subplots(1, 1, figsize=(25, 4))
final_sums.plot(kind='bar', ax=ax)
ax.set_xticklabels(final_sums['datetime'])
ax.set_xlabel('Date')
ax.set_ylabel('Avg Price Difference')
ax.set_title('Days When List Price Exceeded Average List Price for Individual AirBnbs')

plt.show()

ValueError: view limit minimum -36875.96773446315 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1a1a06c9d8> (for post_execute):


ValueError: view limit minimum -36875.96773446315 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -36875.96773446315 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

In [36]:
# Subset the data for easier viewing 

fall = final_sums[(final_sums['datetime'] > '2018-09-22') & (final_sums['datetime'] < '2018-12-22')] 
winter = final_sums[(final_sums['datetime'] > '2018-12-21') | (final_sums['datetime'] < '2018-03-20')] 
spring = final_sums[(final_sums['datetime'] > '2018-03-19') & (final_sums['datetime'] < '2018-06-21')] 
summer = final_sums[(final_sums['datetime'] > '2018-06-20') & (final_sums['datetime'] < '2018-09-23')]

In [37]:
# WINTER

fig, ax = plt.subplots(1, 1, figsize=(25, 4))
winter.plot(kind='bar', ax=ax)
ax.set_xticklabels(winter['datetime'])
ax.set_xlabel('Date')
ax.set_ylabel('Avg Price Difference')
ax.set_title('Average Price Difference in Winter')
plt.show()

ValueError: view limit minimum -36875.96773446315 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1a1a06c9d8> (for post_execute):


ValueError: view limit minimum -36875.96773446315 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -36875.96773446315 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

In [38]:
# SUMMER 

fig, ax = plt.subplots(1, 1, figsize=(25, 4))
summer.plot(kind='bar', ax=ax)
ax.set_xticklabels(summer['datetime'])
ax.set_xlabel('Date')
ax.set_ylabel('Avg Price Difference')
ax.set_title('Average Price Difference in Summer')

ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1a1a06c9d8> (for post_execute):


ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

In [39]:
#FALL

fig, ax = plt.subplots(1, 1, figsize=(25, 4))
fall.plot(kind='bar', ax=ax)
ax.set_xticklabels(fall['datetime'])
ax.set_xlabel('Date')
ax.set_ylabel('Avg Price Difference')
ax.set_title('Average Price Difference in Fall')

ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1a1a06c9d8> (for post_execute):


ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum 0.0 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units